In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import os
import json

plt.style.use("ggplot")

In [7]:
competition = 'asl-signs'

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('..') / 'input'  / 'competition'
    # !pip install -Uqq fastai
else:
    import zipfile, kaggle
    path = Path.home() / '.data' / 'asl-signs'
    if not path.exists():
        path.mkdir(exist_ok=True)
        kaggle.api.competition_download_cli(competition, path=path)
        zipfile.ZipFile(path / f'{competition}.zip').extractall(path)

# Labeled Data

In [8]:
with open(path / 'sign_to_prediction_index_map.json') as f:
    sign_labels = json.load(f)

In [9]:
train = (pd.read_csv(path / 'train.csv')
         .assign(idx=lambda x: x.participant_id.astype(str) + '_' + x.sequence_id.astype(str))
         .set_index('idx')
        )
train.head()

,path,participant_id,sequence_id,sign
idx,,,,
26734_1000035562,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
28656_1000106739,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
16069_100015657,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
25571_1000210073,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
62590_1000240708,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


# 3D Plotly Viz

In [130]:
def create_landmark_lines(frame_landmarks, body_part='hand'):
    empty_row = pd.Series({
        'frame': None, 'row_id': None, 'type': None, 'x': None, 'y': None, 'z': None
    })
    frame_landmarks = frame_landmarks.set_index('landmark_index')
    
    if frame_landmarks.empty:
        line_indices = [None]
    elif body_part == 'hand':
        line_indices = [0, 1, None, 0, 5, None, 0, 17, None, 
                        1, 2, 3, 4, None,
                        5, 6, 7, 8, None,
                        5, 9, None,
                        9, 10, 11, 12, None,
                        9, 13, None,
                        13, 14, 15, 16, None,
                        13, 17, None,
                        17, 18, 19, 20, None
                       ]
    elif body_part == "pose":
        line_indices = [0, 1, 2, 3, 7, None, 
                        0, 4, 5, 6, 7, 8, None,
                        11, 13, 15, 17, 19, 15, 21, None,
                        11, 12, 14, 16, 18, 20, 16, 22, None,
                        11, 23, 25, 27, 31, 29, 27, None,
                        12, 24, 26, 28, 30, 32, 28, None,
                        23, 24, None
               ]
    else:
        raise ValueError()
    lines = pd.DataFrame([frame_landmarks.loc[i, ] if i is not None else empty_row
                          for i in line_indices
                         ])
    return lines

In [232]:
def sample_by_sign(sign, random_state=69):
    mask = train.loc[:, 'sign'] == sign
    sample = train.loc[mask, :].sample(n=1, random_state=random_state)

    landmarks = pd.read_parquet(path / sample['path'].values[0])
    landmarks = landmarks.loc[~landmarks.loc[:, COORD_COLS].isna().all(axis=1), :]
    return sample, landmarks

In [233]:
sample, landmarks = sample_by_sign('bird', 696969)

pose_mask = landmarks.type.isin(['right_hand', 'pose'])
landmarks = landmarks.loc[pose_mask, ]

frames = landmarks.frame.unique().tolist()

rounding_n = 2
(landmarks.groupby('type')
 .agg(
    cnt_landmarks=pd.NamedAgg('landmark_index', 'nunique'),
    cnt_frames=pd.NamedAgg('frame', 'nunique'),
    x_min=pd.NamedAgg('x', 'min'),
    x_max=pd.NamedAgg('x', 'max'),
    y_min=pd.NamedAgg('y', 'min'),
    y_max=pd.NamedAgg('y', 'max'),
    z_min=pd.NamedAgg('z', 'min'),
    z_max=pd.NamedAgg('z', 'max'),
 )
 .assign(
     x_min=lambda x: x.x_min.round(rounding_n),
     x_max=lambda x: x.x_max.round(rounding_n),
     y_min=lambda x: x.y_min.round(rounding_n),
     y_max=lambda x: x.y_max.round(rounding_n),
     z_min=lambda x: x.z_min.round(rounding_n),
     z_max=lambda x: x.z_max.round(rounding_n),
 )
 .T
)

type,pose,right_hand
cnt_landmarks,33.00,21.00
cnt_frames,51.00,51.00
x_min,-0.27,0.02
x_max,1.31,0.59
y_min,0.35,0.38
y_max,2.69,0.84
z_min,-3.82,-0.16
z_max,3.55,0.02


In [234]:
rh_mask = landmarks.type == 'right_hand'
rh_landmarks = landmarks.loc[rh_mask, :]

idx = pd.MultiIndex.from_product(
    [frames, rh_landmarks.landmark_index.unique().tolist()],
    names=['frame', 'landmark_index']
)

rh_landmarks = (rh_landmarks.set_index(['frame', 'landmark_index'])
                .reindex(idx, fill_value=np.nan))
rh_landmarks.loc[:, 0, :].assign(x=lambda x: x.x.interpolate())
rh_landmarks = (rh_landmarks.sort_index(level=[1, 0])
                .assign(x=lambda x: x.groupby('landmark_index').x.apply(lambda g: g.interpolate()),
                        y=lambda x: x.groupby('landmark_index').y.apply(lambda g: g.interpolate()),
                        z=lambda x: x.groupby('landmark_index').z.apply(lambda g: g.interpolate()),
                 )
                .sort_index(level=[0, 1])
                .reset_index()
               )

In [236]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2,     
                    specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}]],
)

rh_mask = landmarks.type == 'right_hand'
for frame in frames:
    # frame_landmarks = landmarks.loc[(landmarks.frame == frame) & rh_mask, :]
    frame_landmarks = rh_landmarks.loc[rh_landmarks.frame == frame, ]
    fig.add_trace(
        go.Scatter3d(
            visible=False, 
            x=frame_landmarks.x, y=frame_landmarks.y, z=frame_landmarks.z, text=frame_landmarks.landmark_index,
            marker=dict(size=8, opacity=0.8),
            mode="markers",
        ),
        row=1, col=1,
    )


for frame in frames:
    # frame_landmarks = landmarks.loc[(landmarks.frame == frame) & rh_mask, :]
    frame_landmarks = rh_landmarks.loc[rh_landmarks.frame == frame, ]
    landmark_lines = create_landmark_lines(frame_landmarks)
    fig.add_trace(
        go.Scatter3d(
            visible=False, 
            x=landmark_lines.x, y=landmark_lines.y, z=landmark_lines.z,
            mode="lines",
            hoverinfo='skip',
        ),
        row=1, col=1,
    )

pose_mask = landmarks.type == 'pose'
for frame in frames:
    frame_landmarks = landmarks.loc[(landmarks.frame == frame) & pose_mask, :]
    fig.add_trace(
        go.Scatter3d(
            visible=False, 
            x=frame_landmarks.x, y=frame_landmarks.y, z=frame_landmarks.z, text=frame_landmarks.landmark_index,
            marker=dict(size=8, opacity=0.8),
            mode="markers",
        ),
        row=1, col=2,
    )

for frame in frames:
    frame_landmarks = landmarks.loc[(landmarks.frame == frame) & pose_mask, :]
    landmark_lines = create_landmark_lines(frame_landmarks, "pose")
    fig.add_trace(
        go.Scatter3d(
            visible=False, 
            x=landmark_lines.x, y=landmark_lines.y, z=landmark_lines.z,
            mode="lines",
            hoverinfo='skip',
        ),
        row=1, col=2,
    )

steps = list()
for i, frame in enumerate(frames):
    step = dict(
        method='update',
        args=[{"visible": [False] * len(frames) * 4},
              {"title": "Slider switched to step: " + str(frame)}],
        label=str(frame),
        value=i
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i + len(frames)] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i + len(frames) * 2] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i + len(frames) * 3] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=1,
    currentvalue={'prefix': "Frame: "},
    pad={'t': 50},
    steps=steps
)]

offset = 0.05
fig.update_layout(
    sliders=sliders,
)

fig.show()